In [1]:
from pandas import read_csv
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import gensim
from sklearn.decomposition import FactorAnalysis
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score,precision_recall_curve, roc_auc_score, f1_score, fbeta_score, auc

pf = read_csv("Lemmatized_corps4.csv", sep=',', header=0).drop('Unnamed: 0', axis='columns')

Using TensorFlow backend.


In [2]:
tokenizer = RegexpTokenizer(r'\w+')

pf["tokens"] = pf["article"].apply(tokenizer.tokenize)
pf.head()

,article,categorie,tokens
0,june busy month mac malware active circulation...,3,"[june, busy, month, mac, malware, active, circ..."
1,a classic suv profile bradleywarren photograph...,2,"[a, classic, suv, profile, bradleywarren, phot..."
2,behold stranger fiction nathan mattise author ...,1,"[behold, stranger, fiction, nathan, mattise, a..."
3,google google lock api access gmail data later...,0,"[google, google, lock, api, access, gmail, dat..."
4,when come action movies i rule thumb—one conce...,1,"[when, come, action, movies, i, rule, thumb, o..."


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pf['tokens'], 
                                                    pf['categorie'], 
                                                    test_size=0.5)

In [4]:
from gensim.models import Word2Vec
X_train_w2v = X_train
X_test_w2v = X_test
w2v_vect = Word2Vec(np.hstack([X_train_w2v, X_test_w2v]), size=300, min_count=10, workers=8)
def text2vec(text):
    """Усредняем векторы слов"""
    vecs = []
    for word in text:
        try:
            vecs.append(w2v_vect[word])
        except KeyError:
            pass
    return np.sum(vecs, axis=0) / len(vecs)

w2v_train = X_train_w2v.apply(text2vec)
w2v_test = X_test_w2v.apply(text2vec)
w2v_train = np.dstack(w2v_train)[0]
w2v_train = np.dstack(w2v_train)[0]
w2v_test = np.dstack(w2v_test)[0]
w2v_test = np.dstack(w2v_test)[0]

/home/disco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [5]:
w2v_vect = Word2Vec(pf['tokens'], size=300, min_count=10, workers=8)
w2v_model = pf['tokens'].apply(text2vec)
w2v_model = np.dstack(w2v_model)[0]
w2v_model = np.dstack(w2v_model)[0]

/home/disco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [8]:
w2v_train.shape

(6894, 600)

Попробуем классифицировать данные с исходной размерностью

In [21]:
a = CatBoostClassifier(n_estimators=800, max_depth=10, task_type="GPU").fit(w2v_train, y_train).predict(w2v_test)

0:	learn: 1.5503502	total: 111ms	remaining: 1m 28s
1:	learn: 1.4916766	total: 208ms	remaining: 1m 23s
2:	learn: 1.4369411	total: 298ms	remaining: 1m 19s
3:	learn: 1.3849994	total: 388ms	remaining: 1m 17s
4:	learn: 1.3409624	total: 480ms	remaining: 1m 16s
5:	learn: 1.2975961	total: 567ms	remaining: 1m 15s
6:	learn: 1.2551817	total: 659ms	remaining: 1m 14s
7:	learn: 1.2156062	total: 747ms	remaining: 1m 13s
8:	learn: 1.1810763	total: 837ms	remaining: 1m 13s
9:	learn: 1.1469204	total: 928ms	remaining: 1m 13s
10:	learn: 1.1133507	total: 1.01s	remaining: 1m 12s
11:	learn: 1.0828421	total: 1.1s	remaining: 1m 12s
12:	learn: 1.0528204	total: 1.19s	remaining: 1m 11s
13:	learn: 1.0251826	total: 1.28s	remaining: 1m 12s
14:	learn: 0.9980306	total: 1.38s	remaining: 1m 12s
15:	learn: 0.9730305	total: 1.49s	remaining: 1m 12s
16:	learn: 0.9488147	total: 1.58s	remaining: 1m 12s
17:	learn: 0.9275957	total: 1.69s	remaining: 1m 13s
18:	learn: 0.9056063	total: 1.78s	remaining: 1m 13s
19:	learn: 0.8846589	to

160:	learn: 0.1865248	total: 15.3s	remaining: 1m
161:	learn: 0.1853469	total: 15.4s	remaining: 1m
162:	learn: 0.1842565	total: 15.5s	remaining: 1m
163:	learn: 0.1832249	total: 15.6s	remaining: 1m
164:	learn: 0.1821821	total: 15.6s	remaining: 1m
165:	learn: 0.1811187	total: 15.7s	remaining: 1m
166:	learn: 0.1800534	total: 15.8s	remaining: 60s
167:	learn: 0.1790235	total: 15.9s	remaining: 59.8s
168:	learn: 0.1780892	total: 16s	remaining: 59.7s
169:	learn: 0.1771599	total: 16.1s	remaining: 59.7s
170:	learn: 0.1761492	total: 16.2s	remaining: 59.6s
171:	learn: 0.1751340	total: 16.3s	remaining: 59.4s
172:	learn: 0.1740997	total: 16.4s	remaining: 59.3s
173:	learn: 0.1731614	total: 16.5s	remaining: 59.2s
174:	learn: 0.1722938	total: 16.5s	remaining: 59s
175:	learn: 0.1714158	total: 16.6s	remaining: 58.9s
176:	learn: 0.1704707	total: 16.7s	remaining: 58.8s
177:	learn: 0.1696246	total: 16.8s	remaining: 58.7s
178:	learn: 0.1686802	total: 16.9s	remaining: 58.6s
179:	learn: 0.1677567	total: 17s	rem

320:	learn: 0.0971379	total: 28.1s	remaining: 42s
321:	learn: 0.0968959	total: 28.2s	remaining: 41.9s
322:	learn: 0.0966651	total: 28.3s	remaining: 41.8s
323:	learn: 0.0963885	total: 28.3s	remaining: 41.6s
324:	learn: 0.0960822	total: 28.4s	remaining: 41.5s
325:	learn: 0.0958341	total: 28.5s	remaining: 41.4s
326:	learn: 0.0955502	total: 28.6s	remaining: 41.3s
327:	learn: 0.0953573	total: 28.6s	remaining: 41.2s
328:	learn: 0.0951135	total: 28.7s	remaining: 41.1s
329:	learn: 0.0948072	total: 28.8s	remaining: 41s
330:	learn: 0.0945062	total: 28.9s	remaining: 40.9s
331:	learn: 0.0942148	total: 28.9s	remaining: 40.8s
332:	learn: 0.0939840	total: 29s	remaining: 40.7s
333:	learn: 0.0936799	total: 29.1s	remaining: 40.6s
334:	learn: 0.0934623	total: 29.2s	remaining: 40.5s
335:	learn: 0.0931661	total: 29.2s	remaining: 40.4s
336:	learn: 0.0929477	total: 29.3s	remaining: 40.3s
337:	learn: 0.0926232	total: 29.4s	remaining: 40.2s
338:	learn: 0.0924104	total: 29.5s	remaining: 40.1s
339:	learn: 0.0921

481:	learn: 0.0665616	total: 39.8s	remaining: 26.2s
482:	learn: 0.0664127	total: 39.8s	remaining: 26.1s
483:	learn: 0.0663227	total: 39.9s	remaining: 26s
484:	learn: 0.0661810	total: 40s	remaining: 26s
485:	learn: 0.0660695	total: 40s	remaining: 25.9s
486:	learn: 0.0659426	total: 40.1s	remaining: 25.8s
487:	learn: 0.0658491	total: 40.2s	remaining: 25.7s
488:	learn: 0.0657572	total: 40.2s	remaining: 25.6s
489:	learn: 0.0656336	total: 40.3s	remaining: 25.5s
490:	learn: 0.0654747	total: 40.4s	remaining: 25.4s
491:	learn: 0.0653837	total: 40.4s	remaining: 25.3s
492:	learn: 0.0652869	total: 40.5s	remaining: 25.2s
493:	learn: 0.0651887	total: 40.6s	remaining: 25.1s
494:	learn: 0.0650683	total: 40.6s	remaining: 25s
495:	learn: 0.0649327	total: 40.7s	remaining: 25s
496:	learn: 0.0647510	total: 40.8s	remaining: 24.9s
497:	learn: 0.0646910	total: 40.9s	remaining: 24.8s
498:	learn: 0.0645495	total: 40.9s	remaining: 24.7s
499:	learn: 0.0643902	total: 41s	remaining: 24.6s
500:	learn: 0.0642673	tota

640:	learn: 0.0501765	total: 50.9s	remaining: 12.6s
641:	learn: 0.0500803	total: 50.9s	remaining: 12.5s
642:	learn: 0.0499910	total: 51s	remaining: 12.5s
643:	learn: 0.0498969	total: 51.1s	remaining: 12.4s
644:	learn: 0.0498371	total: 51.2s	remaining: 12.3s
645:	learn: 0.0497551	total: 51.2s	remaining: 12.2s
646:	learn: 0.0496904	total: 51.3s	remaining: 12.1s
647:	learn: 0.0496185	total: 51.4s	remaining: 12s
648:	learn: 0.0495530	total: 51.4s	remaining: 12s
649:	learn: 0.0494757	total: 51.5s	remaining: 11.9s
650:	learn: 0.0494149	total: 51.6s	remaining: 11.8s
651:	learn: 0.0492902	total: 51.6s	remaining: 11.7s
652:	learn: 0.0492115	total: 51.7s	remaining: 11.6s
653:	learn: 0.0491359	total: 51.8s	remaining: 11.6s
654:	learn: 0.0490461	total: 51.8s	remaining: 11.5s
655:	learn: 0.0489966	total: 51.9s	remaining: 11.4s
656:	learn: 0.0489372	total: 52s	remaining: 11.3s
657:	learn: 0.0488640	total: 52s	remaining: 11.2s
658:	learn: 0.0487840	total: 52.1s	remaining: 11.1s
659:	learn: 0.0486901	

In [22]:
print('Accuracy:', accuracy_score(y_test, a.round()))
print('Precision:', precision_score(y_test, a.round(),average='micro'))           
print('Recall:', recall_score(y_test, a.round(),average='micro'))
print('F1:', f1_score(y_test, a.round(),average='micro'))
print('F0.5:', fbeta_score(y_test, a.round(), 0.5,average='micro'))
print('F2:', fbeta_score(y_test, a.round(), 2,average='micro'))

Accuracy: 0.9415079673532841
Precision: 0.9415079673532841
Recall: 0.9415079673532841
F1: 0.9415079673532841
F0.5: 0.9415079673532841
F2: 0.9415079673532841


Теперь - с уменьшенной размерностью

In [27]:
transformer = FactorAnalysis(n_components=100, random_state=0)
X_transformer = transformer.fit(w2v_model)
X_train_trans = X_transformer.transform(w2v_train)
X_test_trans = X_transformer.transform(w2v_test)
X_train_trans.shape

(5145, 100)

In [28]:
a = CatBoostClassifier(n_estimators=800, max_depth=10, task_type="GPU").fit(X_train_trans, y_train).predict(X_test_trans)

0:	learn: 1.5471905	total: 44.5ms	remaining: 35.5s
1:	learn: 1.4896978	total: 88.5ms	remaining: 35.3s
2:	learn: 1.4390197	total: 129ms	remaining: 34.1s
3:	learn: 1.3897286	total: 166ms	remaining: 33s
4:	learn: 1.3452314	total: 205ms	remaining: 32.6s
5:	learn: 1.3013699	total: 242ms	remaining: 32.1s
6:	learn: 1.2626199	total: 281ms	remaining: 31.8s
7:	learn: 1.2269036	total: 320ms	remaining: 31.7s
8:	learn: 1.1927413	total: 358ms	remaining: 31.5s
9:	learn: 1.1603176	total: 397ms	remaining: 31.3s
10:	learn: 1.1290335	total: 433ms	remaining: 31s
11:	learn: 1.1001812	total: 470ms	remaining: 30.8s
12:	learn: 1.0734275	total: 510ms	remaining: 30.9s
13:	learn: 1.0465859	total: 546ms	remaining: 30.7s
14:	learn: 1.0212852	total: 582ms	remaining: 30.5s
15:	learn: 0.9978289	total: 621ms	remaining: 30.4s
16:	learn: 0.9731833	total: 657ms	remaining: 30.3s
17:	learn: 0.9509522	total: 693ms	remaining: 30.1s
18:	learn: 0.9304222	total: 732ms	remaining: 30.1s
19:	learn: 0.9104643	total: 770ms	remaining

165:	learn: 0.1971839	total: 6.58s	remaining: 25.1s
166:	learn: 0.1960200	total: 6.61s	remaining: 25.1s
167:	learn: 0.1949051	total: 6.65s	remaining: 25s
168:	learn: 0.1937846	total: 6.68s	remaining: 24.9s
169:	learn: 0.1926477	total: 6.72s	remaining: 24.9s
170:	learn: 0.1915635	total: 6.75s	remaining: 24.8s
171:	learn: 0.1904816	total: 6.79s	remaining: 24.8s
172:	learn: 0.1895035	total: 6.83s	remaining: 24.7s
173:	learn: 0.1883740	total: 6.86s	remaining: 24.7s
174:	learn: 0.1874227	total: 6.9s	remaining: 24.6s
175:	learn: 0.1864672	total: 6.93s	remaining: 24.6s
176:	learn: 0.1854042	total: 6.97s	remaining: 24.5s
177:	learn: 0.1844090	total: 7.01s	remaining: 24.5s
178:	learn: 0.1833926	total: 7.04s	remaining: 24.4s
179:	learn: 0.1824513	total: 7.08s	remaining: 24.4s
180:	learn: 0.1815220	total: 7.12s	remaining: 24.3s
181:	learn: 0.1805733	total: 7.15s	remaining: 24.3s
182:	learn: 0.1796327	total: 7.18s	remaining: 24.2s
183:	learn: 0.1786953	total: 7.22s	remaining: 24.2s
184:	learn: 0.1

325:	learn: 0.1017381	total: 12.3s	remaining: 17.9s
326:	learn: 0.1014121	total: 12.3s	remaining: 17.8s
327:	learn: 0.1010806	total: 12.4s	remaining: 17.8s
328:	learn: 0.1007755	total: 12.4s	remaining: 17.8s
329:	learn: 0.1005124	total: 12.5s	remaining: 17.7s
330:	learn: 0.1002099	total: 12.5s	remaining: 17.7s
331:	learn: 0.0998539	total: 12.5s	remaining: 17.7s
332:	learn: 0.0996116	total: 12.6s	remaining: 17.6s
333:	learn: 0.0993183	total: 12.6s	remaining: 17.6s
334:	learn: 0.0989919	total: 12.6s	remaining: 17.5s
335:	learn: 0.0986665	total: 12.7s	remaining: 17.5s
336:	learn: 0.0983249	total: 12.7s	remaining: 17.4s
337:	learn: 0.0980254	total: 12.7s	remaining: 17.4s
338:	learn: 0.0977143	total: 12.7s	remaining: 17.3s
339:	learn: 0.0973921	total: 12.8s	remaining: 17.3s
340:	learn: 0.0970831	total: 12.8s	remaining: 17.3s
341:	learn: 0.0968434	total: 12.8s	remaining: 17.2s
342:	learn: 0.0965452	total: 12.9s	remaining: 17.2s
343:	learn: 0.0962188	total: 12.9s	remaining: 17.1s
344:	learn: 

484:	learn: 0.0688495	total: 17.3s	remaining: 11.2s
485:	learn: 0.0687450	total: 17.3s	remaining: 11.2s
486:	learn: 0.0686349	total: 17.4s	remaining: 11.2s
487:	learn: 0.0684534	total: 17.4s	remaining: 11.1s
488:	learn: 0.0683224	total: 17.4s	remaining: 11.1s
489:	learn: 0.0681954	total: 17.5s	remaining: 11.1s
490:	learn: 0.0680676	total: 17.5s	remaining: 11s
491:	learn: 0.0679138	total: 17.5s	remaining: 11s
492:	learn: 0.0678092	total: 17.6s	remaining: 10.9s
493:	learn: 0.0676788	total: 17.6s	remaining: 10.9s
494:	learn: 0.0675281	total: 17.6s	remaining: 10.9s
495:	learn: 0.0673929	total: 17.7s	remaining: 10.8s
496:	learn: 0.0672518	total: 17.7s	remaining: 10.8s
497:	learn: 0.0670967	total: 17.7s	remaining: 10.8s
498:	learn: 0.0668900	total: 17.8s	remaining: 10.7s
499:	learn: 0.0667488	total: 17.8s	remaining: 10.7s
500:	learn: 0.0665883	total: 17.9s	remaining: 10.7s
501:	learn: 0.0664373	total: 17.9s	remaining: 10.6s
502:	learn: 0.0662680	total: 17.9s	remaining: 10.6s
503:	learn: 0.06

643:	learn: 0.0510044	total: 22.2s	remaining: 5.38s
644:	learn: 0.0509513	total: 22.2s	remaining: 5.35s
645:	learn: 0.0508457	total: 22.3s	remaining: 5.31s
646:	learn: 0.0507492	total: 22.3s	remaining: 5.28s
647:	learn: 0.0506532	total: 22.3s	remaining: 5.24s
648:	learn: 0.0505783	total: 22.4s	remaining: 5.21s
649:	learn: 0.0504680	total: 22.4s	remaining: 5.17s
650:	learn: 0.0504029	total: 22.4s	remaining: 5.13s
651:	learn: 0.0502970	total: 22.5s	remaining: 5.1s
652:	learn: 0.0502509	total: 22.5s	remaining: 5.06s
653:	learn: 0.0501792	total: 22.5s	remaining: 5.03s
654:	learn: 0.0500900	total: 22.6s	remaining: 4.99s
655:	learn: 0.0499995	total: 22.6s	remaining: 4.96s
656:	learn: 0.0499098	total: 22.6s	remaining: 4.92s
657:	learn: 0.0498174	total: 22.6s	remaining: 4.89s
658:	learn: 0.0497390	total: 22.7s	remaining: 4.85s
659:	learn: 0.0496540	total: 22.7s	remaining: 4.82s
660:	learn: 0.0495630	total: 22.7s	remaining: 4.78s
661:	learn: 0.0494397	total: 22.8s	remaining: 4.75s
662:	learn: 0

In [29]:
print('Accuracy:', accuracy_score(y_test, a.round()))
print('Precision:', precision_score(y_test, a.round(),average='micro'))           
print('Recall:', recall_score(y_test, a.round(),average='micro'))
print('F1:', f1_score(y_test, a.round(),average='micro'))
print('F0.5:', fbeta_score(y_test, a.round(), 0.5,average='micro'))
print('F2:', fbeta_score(y_test, a.round(), 2,average='micro'))

Accuracy: 0.9362611737271668
Precision: 0.9362611737271668
Recall: 0.9362611737271668
F1: 0.9362611737271668
F0.5: 0.9362611737271668
F2: 0.9362611737271668


Теперь протестируем другой модуль - он дает более гибкую настройку параметров анализа

In [36]:
!pip install factor_analyzer

     |████████████████████████████████| 40kB 583kB/s eta 0:00:01
  Created wheel for factor-analyzer: filename=factor_analyzer-0.3.2-cp37-none-any.whl size=40382 sha256=1ff09d6b6fcb463599483eacb2e1af6323adae047c7d0f759b0647991c65bff8
  Stored in directory: /home/disco/.cache/pip/wheels/4a/d0/57/f1330cb9c80e82d8d05391c74c94ed61ce3f03bf6157f3d6db
Successfully built factor-analyzer


In [6]:
from factor_analyzer import FactorAnalyzer

fac = FactorAnalyzer(n_factors=100, rotation = 'promax', method = 'ml')

In [7]:
fac_trans = fac.fit(w2v_model)

/home/disco/anaconda3/lib/python3.7/site-packages/factor_analyzer/factor_analyzer.py:536: UserWarning: Failed to converge: b'STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT'
  warnings.warn('Failed to converge: {}'.format(res.message))


In [8]:
X_train_trans = fac_trans.transform(w2v_train)
X_test_trans = fac_trans.transform(w2v_test)
X_train_trans.shape

(5145, 100)

In [9]:
a = CatBoostClassifier(n_estimators=800, max_depth=10, task_type="GPU").fit(X_train_trans, y_train).predict(X_test_trans)

0:	learn: 1.5499833	total: 108ms	remaining: 1m 26s
1:	learn: 1.4958704	total: 152ms	remaining: 1m
2:	learn: 1.4492680	total: 198ms	remaining: 52.6s
3:	learn: 1.4027323	total: 241ms	remaining: 47.9s
4:	learn: 1.3615157	total: 286ms	remaining: 45.5s
5:	learn: 1.3184089	total: 329ms	remaining: 43.6s
6:	learn: 1.2804281	total: 371ms	remaining: 42s
7:	learn: 1.2464938	total: 413ms	remaining: 40.9s
8:	learn: 1.2144005	total: 457ms	remaining: 40.2s
9:	learn: 1.1803070	total: 498ms	remaining: 39.3s
10:	learn: 1.1507818	total: 539ms	remaining: 38.7s
11:	learn: 1.1211739	total: 578ms	remaining: 38s
12:	learn: 1.0938245	total: 617ms	remaining: 37.4s
13:	learn: 1.0666644	total: 656ms	remaining: 36.8s
14:	learn: 1.0397862	total: 691ms	remaining: 36.2s
15:	learn: 1.0156884	total: 728ms	remaining: 35.7s
16:	learn: 0.9937388	total: 768ms	remaining: 35.4s
17:	learn: 0.9701862	total: 802ms	remaining: 34.8s
18:	learn: 0.9488528	total: 839ms	remaining: 34.5s
19:	learn: 0.9289870	total: 876ms	remaining: 34

161:	learn: 0.2083968	total: 6.3s	remaining: 24.8s
162:	learn: 0.2072292	total: 6.34s	remaining: 24.8s
163:	learn: 0.2060401	total: 6.38s	remaining: 24.7s
164:	learn: 0.2048689	total: 6.42s	remaining: 24.7s
165:	learn: 0.2037746	total: 6.47s	remaining: 24.7s
166:	learn: 0.2026876	total: 6.51s	remaining: 24.7s
167:	learn: 0.2016220	total: 6.55s	remaining: 24.6s
168:	learn: 0.2004845	total: 6.58s	remaining: 24.6s
169:	learn: 0.1993449	total: 6.62s	remaining: 24.5s
170:	learn: 0.1982693	total: 6.67s	remaining: 24.5s
171:	learn: 0.1972192	total: 6.71s	remaining: 24.5s
172:	learn: 0.1962700	total: 6.75s	remaining: 24.5s
173:	learn: 0.1952143	total: 6.79s	remaining: 24.4s
174:	learn: 0.1941261	total: 6.83s	remaining: 24.4s
175:	learn: 0.1933041	total: 6.88s	remaining: 24.4s
176:	learn: 0.1922845	total: 6.92s	remaining: 24.4s
177:	learn: 0.1912350	total: 6.96s	remaining: 24.3s
178:	learn: 0.1902380	total: 6.99s	remaining: 24.3s
179:	learn: 0.1891757	total: 7.03s	remaining: 24.2s
180:	learn: 0

326:	learn: 0.1073192	total: 12.2s	remaining: 17.6s
327:	learn: 0.1070703	total: 12.2s	remaining: 17.6s
328:	learn: 0.1068037	total: 12.3s	remaining: 17.5s
329:	learn: 0.1064737	total: 12.3s	remaining: 17.5s
330:	learn: 0.1061980	total: 12.3s	remaining: 17.4s
331:	learn: 0.1058897	total: 12.3s	remaining: 17.4s
332:	learn: 0.1056737	total: 12.4s	remaining: 17.4s
333:	learn: 0.1053454	total: 12.4s	remaining: 17.3s
334:	learn: 0.1050638	total: 12.4s	remaining: 17.3s
335:	learn: 0.1047944	total: 12.5s	remaining: 17.2s
336:	learn: 0.1044534	total: 12.5s	remaining: 17.2s
337:	learn: 0.1041920	total: 12.5s	remaining: 17.1s
338:	learn: 0.1038993	total: 12.6s	remaining: 17.1s
339:	learn: 0.1036106	total: 12.6s	remaining: 17.1s
340:	learn: 0.1033452	total: 12.6s	remaining: 17s
341:	learn: 0.1030775	total: 12.7s	remaining: 17s
342:	learn: 0.1027880	total: 12.7s	remaining: 16.9s
343:	learn: 0.1025449	total: 12.7s	remaining: 16.9s
344:	learn: 0.1022213	total: 12.8s	remaining: 16.8s
345:	learn: 0.10

485:	learn: 0.0724981	total: 17.2s	remaining: 11.1s
486:	learn: 0.0723599	total: 17.2s	remaining: 11.1s
487:	learn: 0.0722391	total: 17.3s	remaining: 11s
488:	learn: 0.0720791	total: 17.3s	remaining: 11s
489:	learn: 0.0718984	total: 17.3s	remaining: 11s
490:	learn: 0.0717372	total: 17.4s	remaining: 10.9s
491:	learn: 0.0715462	total: 17.4s	remaining: 10.9s
492:	learn: 0.0713990	total: 17.4s	remaining: 10.8s
493:	learn: 0.0712430	total: 17.4s	remaining: 10.8s
494:	learn: 0.0710992	total: 17.5s	remaining: 10.8s
495:	learn: 0.0709019	total: 17.5s	remaining: 10.7s
496:	learn: 0.0707693	total: 17.5s	remaining: 10.7s
497:	learn: 0.0706866	total: 17.6s	remaining: 10.7s
498:	learn: 0.0705513	total: 17.6s	remaining: 10.6s
499:	learn: 0.0703903	total: 17.6s	remaining: 10.6s
500:	learn: 0.0702215	total: 17.7s	remaining: 10.5s
501:	learn: 0.0700647	total: 17.7s	remaining: 10.5s
502:	learn: 0.0699312	total: 17.7s	remaining: 10.5s
503:	learn: 0.0697986	total: 17.8s	remaining: 10.4s
504:	learn: 0.0696

646:	learn: 0.0531360	total: 22.1s	remaining: 5.22s
647:	learn: 0.0530568	total: 22.1s	remaining: 5.18s
648:	learn: 0.0530022	total: 22.1s	remaining: 5.15s
649:	learn: 0.0529108	total: 22.2s	remaining: 5.11s
650:	learn: 0.0528188	total: 22.2s	remaining: 5.08s
651:	learn: 0.0527298	total: 22.2s	remaining: 5.04s
652:	learn: 0.0526354	total: 22.3s	remaining: 5.01s
653:	learn: 0.0525672	total: 22.3s	remaining: 4.98s
654:	learn: 0.0524804	total: 22.3s	remaining: 4.94s
655:	learn: 0.0523778	total: 22.4s	remaining: 4.91s
656:	learn: 0.0523203	total: 22.4s	remaining: 4.87s
657:	learn: 0.0522305	total: 22.4s	remaining: 4.84s
658:	learn: 0.0521595	total: 22.4s	remaining: 4.8s
659:	learn: 0.0520504	total: 22.5s	remaining: 4.77s
660:	learn: 0.0519764	total: 22.5s	remaining: 4.73s
661:	learn: 0.0518908	total: 22.5s	remaining: 4.7s
662:	learn: 0.0518137	total: 22.6s	remaining: 4.66s
663:	learn: 0.0517289	total: 22.6s	remaining: 4.63s
664:	learn: 0.0516513	total: 22.6s	remaining: 4.59s
665:	learn: 0.

In [10]:
print('Accuracy:', accuracy_score(y_test, a.round()))
print('Precision:', precision_score(y_test, a.round(),average='micro'))           
print('Recall:', recall_score(y_test, a.round(),average='micro'))
print('F1:', f1_score(y_test, a.round(),average='micro'))
print('F0.5:', fbeta_score(y_test, a.round(), 0.5,average='micro'))
print('F2:', fbeta_score(y_test, a.round(), 2,average='micro'))

Accuracy: 0.9282938204430625
Precision: 0.9282938204430625
Recall: 0.9282938204430625
F1: 0.9282938204430625
F0.5: 0.9282938204430624
F2: 0.9282938204430624


И, наконец, самое интересное - нейросеть. По факту при изменении размерности между слоями нейросети, проводится факторный анализ, однако важное отличие в том, что практически невозможно интерпретировать промежуточные результаты работы нейросети.

In [70]:
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 35
VOCAB_SIZE = len(VOCAB)

VALIDATION_SPLIT=.2
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(pf["article"].tolist())
sequences = tokenizer.texts_to_sequences(pf["article"].tolist())

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

cnn_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 119771 unique tokens.


In [71]:
all_words = [word for tokens in pf["tokens"] for word in tokens]
sentence_lengths = [len(tokens) for tokens in pf["tokens"]]
VOCAB = sorted(list(set(all_words)))

word2vec_path = "GoogleNews-vectors-negative300.bin.gz"
#https://www.kaggle.com/leadbest/googlenewsvectorsnegative300
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)
embedding_weights = np.zeros((len(word_index)+1, EMBEDDING_DIM))

for word,index in word_index.items():
    embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(embedding_weights.shape)

(119772, 300)


In [72]:
labels = to_categorical(np.asarray(pf["categorie"]), num_classes=5, dtype='float64')

In [73]:
indices = np.arange(cnn_data.shape[0])
np.random.shuffle(indices)
cnn_data = cnn_data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * cnn_data.shape[0])

In [74]:
from keras.layers import Dense, Input, Flatten, Dropout, concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Model

def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index, trainable=False, extra_conv=True):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    l_merge = concatenate(convs, axis=1)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    #x = Dropout(0.5)(x)

    preds = Dense(labels_index, activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])

    return model

In [75]:
model = ConvNet(embedding_weights, MAX_SEQUENCE_LENGTH, len(word_index)+1, EMBEDDING_DIM, 
                len(list(pf["categorie"].unique())))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [76]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 35)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 35, 300)      35931600    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 33, 128)      115328      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 32, 128)      153728      embedding_1[0][0]                
____________________________________________________________________________________________

In [84]:
x_train = cnn_data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_test = cnn_data[-num_validation_samples:]
y_test = labels[-num_validation_samples:]

model.fit(x_train, y_train, 
          validation_data=(x_test, y_test), 
          epochs=5, 
          batch_size=256)

Train on 8233 samples, validate on 2058 samples
Epoch 1/5
8233/8233 [==============================] - 3s 309us/step - loss: 0.0279 - acc: 0.9921 - val_loss: 0.8523 - val_acc: 0.8241
Epoch 2/5
8233/8233 [==============================] - 3s 305us/step - loss: 0.0225 - acc: 0.9931 - val_loss: 0.8834 - val_acc: 0.8246
Epoch 3/5
8233/8233 [==============================] - 2s 299us/step - loss: 0.0160 - acc: 0.9957 - val_loss: 0.8450 - val_acc: 0.8280
Epoch 4/5
8233/8233 [==============================] - 2s 299us/step - loss: 0.0134 - acc: 0.9968 - val_loss: 0.8507 - val_acc: 0.8309
Epoch 5/5
8233/8233 [==============================] - 2s 299us/step - loss: 0.0098 - acc: 0.9978 - val_loss: 0.8610 - val_acc: 0.8241
